# Cross-Feeding Simulation
Here we will simulate cross-feeding between two species of bacteria, inspired by previous work by [Dukovski et al., 2021](https://www.nature.com/articles/s41596-021-00593-3). For this purpose we will simulate two strains of *E. coli*: one which cannot uptake lactose and one which cannot metabolize galactose - with lactose being the only environmental carbon source being lactose set to a StaticConcentration.

In [1]:
from copy import deepcopy

import cobra.io
from cobra.io import write_sbml_model

from process_bigraph import Composite
from process_bigraph.composite import ProcessTypes

from cdFBA import register_types
from cdFBA.processes.dfba import DFBA, UpdateEnvironment, StaticConcentration, Injector, WaveFunction

from cdFBA.utils import make_cdfba_composite, get_injector_spec, get_wave_spec, get_static_spec, model_from_file

from matplotlib import pyplot as plt
from pprint import pprint

import cProfile
import pstats
import io


<unknown>:1: SyntaxWarning: invalid escape sequence '\s'


## Create Models
First we create E.coli models and save them in the sbml format. We start off with the `iAF1260` model for the K-12 *E. coli* strain, and use the knockout andCOBRApy

In [2]:
#Load E. coli model
E_no_galE = cobra.io.load_model("iAF1260")
#Copy model
E_no_LCTStex = E_no_galE.copy()
#Knock out galactose metabolism
E_no_galE.genes.b0759.knock_out() # cannot metabolize galactose
#Knock out Lactose transport reaction LCTStex
E_no_LCTStex.reactions.LCTStex.knock_out() # cannot uptake lactose

In [3]:
#save models in SBML format
# write_sbml_model(E_no_galE, "/Users/tasnifrahman/PycharmProjects/cdFBA/Notebooks/E_no_galE.xml")
# write_sbml_model(E_no_LCTStex, "/Users/tasnifrahman/PycharmProjects/cdFBA/Notebooks/E_no_LCTStex.xml")

In [4]:
#create model dictionary
model_dict = {
    'deltaGal':'/Users/tasnifrahman/PycharmProjects/cdFBA/Notebooks/E_no_galE.xml',
    'deltaLac':'/Users/tasnifrahman/PycharmProjects/cdFBA/Notebooks/E_no_LCTStex.xml'
}
#list exchange reactions
exchanges = []

In [ ]:
model = model_from_file('/Users/tasnifrahman/Research/cdFBA Test/E_no_galE.xml')